In [0]:
#Pysaprk-DecisionTreeClasssifer
#Import the required libraries
from __future__ import print_function
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer,StringIndexer,StandardScaler,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
#Read or load the dataset
df1 = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/praneethsanthosh555@gmail.com/diabetes-3.csv",header=True)

In [0]:
#print first five rows of the data
df1.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
 6| 148| 72| 35| 0|33.6| 0.627| 50| 1|
 1| 85| 66| 29| 0|26.6| 0.351| 31| 0|
 8| 183| 64| 0| 0|23.3| 0.672| 32| 1|
 1| 89| 66| 23| 94|28.1| 0.167| 21| 0|
 0| 137| 40| 35| 168|43.1| 2.288| 33| 1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows

In [0]:
df1.printSchema()

root
-- Pregnancies: string (nullable = true)
-- Glucose: string (nullable = true)
-- BloodPressure: string (nullable = true)
-- SkinThickness: string (nullable = true)
-- Insulin: string (nullable = true)
-- BMI: string (nullable = true)
-- DiabetesPedigreeFunction: string (nullable = true)
-- Age: string (nullable = true)
-- Outcome: string (nullable = true)

In [0]:
#Get the data in sql
from pyspark.sql.functions import col, when , count
new_data=df1.select(*(col(c).cast('float').alias(c) for c in df1.columns))
new_data.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction| Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+
 6.0| 148.0| 72.0| 35.0| 0.0|33.6| 0.627|50.0| 1.0|
 1.0| 85.0| 66.0| 29.0| 0.0|26.6| 0.351|31.0| 0.0|
 8.0| 183.0| 64.0| 0.0| 0.0|23.3| 0.672|32.0| 1.0|
 1.0| 89.0| 66.0| 23.0| 94.0|28.1| 0.167|21.0| 0.0|
 0.0| 137.0| 40.0| 35.0| 168.0|43.1| 2.288|33.0| 1.0|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+
only showing top 5 rows

In [0]:
#Check any missing values are present in the data
miss_data=new_data.select([count(when(col(c).isNull(),c)).alias(c) for c in new_data.columns])
miss_data.show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+

In [0]:
#Split the data in to separete features
col=new_data.columns
col.remove("Outcome")

In [0]:
#Assemble all the data
assemble_data=VectorAssembler(inputCols=col,outputCol="features")
assembler=assemble_data.transform(new_data)

data=assembler.select("features","Outcome")
data.show(5,truncate=False)

+----------------------------------------------------------------------+-------+
features |Outcome|
+----------------------------------------------------------------------+-------+
[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0] |1.0 |
[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0] |0.0 |
[8.0,183.0,64.0,0.0,0.0,23.299999237060547,0.671999990940094,32.0] |1.0 |
[1.0,89.0,66.0,23.0,94.0,28.100000381469727,0.16699999570846558,21.0] |0.0 |
[0.0,137.0,40.0,35.0,168.0,43.099998474121094,2.2880001068115234,33.0]|1.0 |
+----------------------------------------------------------------------+-------+
only showing top 5 rows

In [0]:
#Splitting the data for thr training set and testing set with the randomsplit of 0.7 & 0.3
train_df,test_df=data.randomSplit([0.7,0.3])
#Print the train and test data after splitting
print(train_df.show(5))
print(test_df.show(5))

+--------------------+-------+
 features|Outcome|
+--------------------+-------+
(8,[0,1,6,7],[2.0...| 0.0|
(8,[0,1,6,7],[3.0...| 0.0|
(8,[0,1,6,7],[6.0...| 0.0|
(8,[0,1,6,7],[7.0...| 0.0|
(8,[0,1,6,7],[10....| 1.0|
+--------------------+-------+
only showing top 5 rows

None
+--------------------+-------+
 features|Outcome|
+--------------------+-------+
(8,[0,1,6,7],[2.0...| 0.0|
(8,[1,5,6,7],[73....| 0.0|
(8,[1,5,6,7],[131...| 1.0|
(8,[1,5,6,7],[138...| 1.0|
[0.0,57.0,60.0,0....| 0.0|
+--------------------+-------+
only showing top 5 rows

None

In [0]:
#Let's build a model with respect to DecisionTreeClassifer
dt=DecisionTreeClassifier(labelCol="Outcome",featuresCol="features")
model=dt.fit(train_df)

In [0]:
#Get the predictions on the test data
predictions=model.transform(test_df)
predictions.select("Outcome","prediction").show(5)

+-------+----------+
Outcome|prediction|
+-------+----------+
 0.0| 0.0|
 0.0| 0.0|
 1.0| 1.0|
 1.0| 1.0|
 0.0| 0.0|
+-------+----------+
only showing top 5 rows

In [0]:
#print the performance of the DecisionTreeClassifer on our test data 
evaluation=MulticlassClassificationEvaluator(labelCol="Outcome",predictionCol="prediction",
                                            metricName="accuracy")
predictions_text=evaluation.evaluate(predictions)
print("The accuracy is found out to be: %s" % predictions_text)

The accuracy is found out to be: 0.7574468085106383

In [0]:
evaluation_roc=MulticlassClassificationEvaluator(labelCol="Outcome",predictionCol="prediction",metricName="hammingLoss")
prediction_roc_auc=evaluation_roc.evaluate(predictions)
print('The Hamming loss score is : %d' % prediction_roc_auc)

The Hamming loss score is : 0